# Data processing

In [56]:
import pandas as pd
import json
from os import walk
def collect_ind(sub_class):
    filenames = next(walk(f'data/{sub_class}'), (None, None, []))[2]  # [] if no file

    ind_all = []
    for i in range(len(filenames)):
        recipe = filenames[i]
        # read file
        with open(f'data/{sub_class}/{recipe}', 'r') as myfile:
            rec=myfile.read()
        js_rec = json.loads(rec)
        ind_all+=js_rec['ingredients']
        
    ind_all = pd.DataFrame(ind_all).rename(columns={'name':'ingredients'})
    
    return ind_all

def count_condiment(data,name):
    data = (
        pd.DataFrame(data.ingredients.value_counts())
    .reset_index(drop=False)
    .rename(columns = {'index':'ingredients','ingredients':name})
    .merge(ind_class,on = 'ingredients',how = 'left')
    .fillna('condiments(exclusive)')
    .drop('ingredients',axis = 1)
    )
    
    data = pd.DataFrame(data.type.value_counts()).reset_index(drop = False).rename(columns ={'index':'type','type':name})
    return data

ind_class = pd.read_excel('503-ingredients.xlsx')
ind_class = ind_class.melt(value_name='ingredients',var_name = 'type')
ind_class = ind_class.dropna()
with open('all_ingredients.txt') as file:
    lines = file.readlines()

# eval(lines[0])
# data = []

# for i in lines:
#     data.append(eval(i))
# data = pd.DataFrame(data,columns=['ingredients','counts'])
# data = data.merge(ind_class,on = 'ingredients',how = 'left')
# data = data.fillna('condiments(exclusive)')

appetizer = collect_ind('appetizer')
beverage = collect_ind('beverage')
breakfast = collect_ind('breakfast')
dessert = collect_ind('dessert')
entree = collect_ind('entree')
salad = collect_ind('salad')
side = collect_ind('side')
soup_stew = collect_ind('soup-stew')

appetizer = count_condiment(appetizer,'appetizer')
beverage = count_condiment(beverage,'beverage')
breakfast = count_condiment(breakfast,'breakfast')
dessert = count_condiment(dessert,'dessert')
entree = count_condiment(entree,'entree')
salad = count_condiment(salad,'salad')
side = count_condiment(side,'side')
soup_stew = count_condiment(soup_stew,'soup_stew')

data = appetizer.merge(beverage,how = 'outer',on = 'type')
data = data.merge(breakfast,how = 'outer',on = 'type')
data = data.merge(dessert,how = 'outer',on = 'type')
data = data.merge(entree,how = 'outer',on = 'type')
data = data.merge(salad,how = 'outer',on = 'type')
data = data.merge(side,how = 'outer',on = 'type')
data = data.merge(soup_stew,how = 'outer',on = 'type')
data = data.fillna(0)

data.iloc[6,0] = 'carbohydrates'

In [57]:
data.to_csv('windrose.csv',index=False)

# plotting

In [12]:
import dash
from dash import dcc,html
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output

In [64]:
data = pd.read_csv('windrose.csv')

In [65]:
data.iloc[:,1:] = data.iloc[:,1:]/data.iloc[:,1:].sum()
data = data.drop(0)

# dash

In [13]:
df = px.data.stocks()

app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Styling using html components', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),

        dcc.Dropdown( id = 'dropdown',
        options = [
            {'label':'appetizer', 'value':'appetizer' },
            ],
        value = 'appetizer'),
        dcc.Graph(id = 'bar_plot')
    ])
    
    
@app.callback(Output(component_id='bar_plot', component_property= 'figure'),
              [Input(component_id='dropdown', component_property= 'value')])
def graph_update(dropdown_value):
    print(dropdown_value)
    fig = px.line_polar(data, r=dropdown_value, theta="type", template="plotly_dark",
                   color_discrete_sequence= px.colors.sequential.Plasma_r,line_close=True)
    
    fig.update_layout(title = dropdown_value
                      )
    return fig  

# app.run_server()

# windrose

In [58]:
import plotly.graph_objects as go

In [66]:
data

,type,appetizer,beverage,breakfast,dessert,entree,salad,side,soup_stew
1,vegetable,0.131868,0.010753,0.080,0.007968,0.085139,0.173913,0.171296,0.138889
2,protein,0.131868,0.010753,0.064,0.027888,0.086687,0.080745,0.069444,0.087302
3,condiment (liquid),0.115385,0.086022,0.104,0.075697,0.060372,0.093168,0.069444,0.079365
4,condiment (powder),0.093407,0.043011,0.112,0.083665,0.054180,0.074534,0.097222,0.063492
5,condiment (bulk),0.065934,0.053763,0.024,0.023904,0.041796,0.062112,0.041667,0.059524
6,carbohydrates,0.049451,0.021505,0.080,0.059761,0.044892,0.043478,0.064815,0.071429
7,processed food,0.043956,0.010753,0.064,0.011952,0.023220,0.024845,0.023148,0.031746
8,beverage,0.032967,0.150538,0.056,0.039841,0.021672,0.037267,0.037037,0.043651
9,other,0.027473,0.000000,0.048,0.003984,0.007740,0.012422,0.018519,0.011905
10,fruit,0.021978,0.043011,0.048,0.047809,0.021672,0.024845,0.009259,0.000000


In [78]:
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=data['appetizer'],
      theta=data["type"],
      fill='toself',
      name='appetizer'
))
fig.add_trace(go.Scatterpolar(
      r=data['breakfast'],
      theta=data["type"],
      fill='toself',
      name='breakfast'
))
fig.add_trace(go.Scatterpolar(
      r=data['beverage'],
      theta=data["type"],
      fill='toself',
      name='beverage'
))
fig.add_trace(go.Scatterpolar(
      r=data['dessert'],
      theta=data["type"],
      fill='toself',
      name='dessert'
))
fig.add_trace(go.Scatterpolar(
      r=data['entree'],
      theta=data["type"],
      fill='toself',
      name='entree'
))
fig.add_trace(go.Scatterpolar(
      r=data['salad'],
      theta=data["type"],
      fill='toself',
      name='salad'
))
fig.add_trace(go.Scatterpolar(
      r=data['soup_stew'],
      theta=data["type"],
      fill='toself',
      name='soup_stew'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      # range=[0, 1]
    )),
  showlegend=True,
  template='ggplot2',
  title = 'Condiment radar plot of each subclass'
)

fig.show()
fig.write_html("windrose.html")